<a href="https://colab.research.google.com/github/aso1801032/AI_tech2020/blob/master/2020AI0305_fishvideo_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##魚の画像を学習して動画分析に利用する

###魚の画像をダウンロードして解凍

In [5]:
# 画像zipのダウンロード
import urllib.request as req

# 魚が写っている画像
url = "https://github.com/masatokg/sample_photo/raw/master/fish.zip"
save_file = "fish.zip"
req.urlretrieve(url,save_file)

# 魚が映っていない画像
url = "https://github.com/masatokg/sample_photo/raw/master/nofish.zip"
save_file = "nofish.zip"
req.urlretrieve(url,save_file)

# それぞれ解凍する
!unzip -o -q "./fish.zip"
!unzip -o -q "./nofish.zip"

###魚の学習のための定義

In [6]:
# ライブラリのinport
import cv2
import os, glob, pickle
from sklearn.model_selection import train_test_split
from sklearn import datasets, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [7]:
# 画像のサイズやパスを指定
image_size = (64,32)
path = "/content"
path_fish = path + "/fish"
path_nofish = path + "/nofish"
x = []# 画像データ（説明変数）用配列
y = []# ラベルデータ（目的変数）用配列
print(path_fish)
print(path_nofish)

/content/fish
/content/nofish


###学習に使う自作関数を定義

In [8]:
# 画像データを読み込んで説明変数、目的変数に追加する処理の関数を定義
def read_dir(path, label,x,y,image_size):
  # 正規表現にマッチするファイルパスリストを取得する
  files = glob.glob(path + "/*.jpg")
  print(files)
  # ファイルパスリストの要素１つずつループ
  for f in files:
    # 画像データを読み込む
    
    img = cv2.imread(f)
    img = cv2.resize(img, image_size) # 学習用にサイズを統一
    img_data = img.reshape(-1,) # 行数任意のいち次元配列に変換
    x.append( img_data ) # 画像データを説明変数配列に追加
    y.append( label ) # 画像データと同じ順番１でラベル値を目的変数に追加

###学習処理

In [9]:
# 画像データを読み込む
read_dir( path_nofish, 0, x, y, image_size ) # ラベルを０：Falseとして登録
read_dir( path_fish, 1, x, y, image_size ) # ラベルを1：Trueとして登録

# データを学習用とテストように分割する
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

#　ランダムフォレストアルゴリズムで学習モデルを生成、学習させる
clf = RandomForestClassifier()
clf.fit( x_train, y_train )

# 精度の確認
y_pred = clf.predict( x_test )
print("学習精度:",accuracy_score(y_test,y_pred))

# 学習モデルのメモリ状態をファイル保存
with open("fish.pkl","wb") as fp:
  pickle.dump(clf, fp)

['/content/nofish/23.jpg', '/content/nofish/22.jpg', '/content/nofish/124.jpg', '/content/nofish/3631.jpg', '/content/nofish/3652.jpg', '/content/nofish/39.jpg', '/content/nofish/226.jpg', '/content/nofish/154.jpg', '/content/nofish/3461.jpg', '/content/nofish/3447.jpg', '/content/nofish/30.jpg', '/content/nofish/21.jpg', '/content/nofish/3593.jpg', '/content/nofish/278.jpg', '/content/nofish/3456.jpg', '/content/nofish/4033.jpg', '/content/nofish/8.jpg', '/content/nofish/7.jpg', '/content/nofish/20.jpg', '/content/nofish/5.jpg', '/content/nofish/18.jpg', '/content/nofish/238.jpg', '/content/nofish/279.jpg', '/content/nofish/132.jpg', '/content/nofish/12.jpg', '/content/nofish/3564.jpg', '/content/nofish/271.jpg', '/content/nofish/3513.jpg', '/content/nofish/266.jpg', '/content/nofish/3448.jpg', '/content/nofish/337.jpg', '/content/nofish/313.jpg', '/content/nofish/33.jpg', '/content/nofish/3454.jpg', '/content/nofish/254.jpg', '/content/nofish/43.jpg', '/content/nofish/3648.jpg', '/co

##学習モデルを利用して動画から魚の画像を検出

###解析の準備

In [10]:
import cv2, os, copy, pickle
from google.colab.patches import cv2_imshow

# 学習済みモデルファイルをプログラムに読み込む
with open("fish.pkl", "rb") as fp:
  clf = pickle.load( fp )
  # with 終わり

  # 解析画像出力ようディレクトリ用文字列
  output_dir = "./bestshot"
  # 解析画像出力ディレクトリが存在しなければ作成
  if not os.path.isdir(output_dir): os.mkdir(output_dir)
  # 　　以下でも同じことができる
  #os.makedirs( output_dir, exist_ok=True )

  # 各処理よう変数を定義
  img_last = None # 前回の画像データよう変数
  fish_th = 3 # 画像を出力するかどうかの閾値（1枚あたりの魚の検出数）
  count = 0
  frame_count = 0

###動画ファイルの読み込み処理

In [11]:
url = "https://github.com/masatokg/sample_photo/raw/master/fish.mp4"
save_file = "fish.mp4"
req.urlretrieve( url, save_file )

('fish.mp4', <http.client.HTTPMessage at 0x7facc3554240>)

###動画をプログラムに読み込み、フレームワークの差分により変化を検出、検出した部分に魚がいるかを推論して、魚がいると判定したらフレームをbestshotディレクトリファイル保存

In [12]:
# openCVで動画データを読み込み
cap = cv2.VideoCapture( save_file )
# フレームがなくなるまで、無限ループ
while True:
  is_ok, frame = cap.read()
  if not is_ok:
    print("end")
    break
    # if分の終わり（初回判定のif文）
  frame = cv2.resize( frame, (640, 360) )# 差分検出対象フレームのサイズを統一
  # 検出エリア枠表示ようの画像データを作る
  frame2 = copy.copy( frame )
  frame_count += 1
  # 前のフレームと比較するために
  gray = cv2.cvtColor( frame,cv2.COLOR_BGR2GRAY )
  gray=cv2.GaussianBlur(gray,(15,15),0)
  img_b = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY )[1]
  # 　前の画像がない（=最初の画像）出なければ実行
  if not img_last is None:
    # 差分を得る
    frame_diff = cv2.absdiff( img_last, img_b )
    cnts = cv2.findContours( frame_diff,
                            cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[0]
    #　各差分領域に魚が写っているか一つずつ調べる
    fish_count = 0
    for pt in cnts:
      #検出した領域情報を変数に格納
      x,y,w,h = cv2.boundingRect(pt)
      if w < 100 or w > 500: continue # ノイズとして無視
      # 抽出した差分領域に魚が映っているか推論するために画像サイズを揃える
      imgex=frame[y:y+h,x:x+w]
      imgex = cv2.resize( imgex, (64, 32) ) # 学習モデルのサイズにあわっせる
      image_data = imgex.reshape( -1, )# 推論のために一次元配列に変換
      # 推論させる
      pred_y = clf.predict([image_data])
      # 魚と判定できたら領域に枠をつけてframe2という画像にする
      if pred_y[0] == 1:
        fish_count += 1
        cv2.rectangle(frame2,(x,y), (x+w, y+h), (0,255,0), 2 ) # Greenの枠
      #if分の終わり
    #for 分の終わり
    # 閾値以上に魚の数が写っていれば、画像ファイルとして出力
    if fish_count > fish_th:
      fname = output_dir + "/fish" + str(count) + ".jpg"
      cv2.imwrite(fname, frame) # 指定した画像名でフレームを保存
      count += 1 # 画像ファイルカウンタを+1
    # if分終わり
  # 負荷が重いのでコメントアウト
  # cv2.imshow(frame2)  枠付きの画像を表示する 
  img_last = img_b # 前回のフレーム画像を記憶

  # while終わり
cap.release() # 動画ファイルを開放
# 出力した枚数を表示
print("ok",count,"/",frame_count, "/枚検出")

end
ok 91 / 1989 /枚検出
